In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# 加载数据
with open('index2.pkl', 'rb') as f:
    dp_index_results = pickle.load(f)

groups = sorted(dp_index_results.keys())
num_groups = len(groups)
group_cmap = cm.get_cmap('plasma_r', num_groups)
traj_cmap = cm.get_cmap('tab10')  # 每组最多10条轨迹，颜色区别明显

# 创建子图
fig, axes = plt.subplots(num_groups, 1, figsize=(8, 2.5 * num_groups), sharex=True)

if num_groups == 1:
    axes = [axes]

for i, group in enumerate(groups):
    ax = axes[i]
    curves = dp_index_results[group]

    for j, curve in enumerate(curves):
        color = traj_cmap(j % 10)  # 每条轨迹不同颜色
        ax.plot(curve, color=color, linewidth=2, label=f"run{j+1}")

    ax.set_ylabel(f"$\epsilon_{{HD}} = {group}$", fontsize=16)
    ax.set_ylim(991, 1083)
    ax.tick_params(axis='both', which='both', direction='in', labelsize=14)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend(fontsize=10, loc='upper right', ncol=2, frameon=False)

axes[-1].set_xlabel("Step Index", fontsize=16)
plt.tight_layout()
#plt.savefig('dp_index_subplots_colored.png', dpi=300)
plt.show()


In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

def extract_valid_segments(dp_index_results, lower_bound=1015, upper_bound=1064, min_length=1000):
    """
    从每个组的每条轨迹中提取出连续的、残基编号在指定范围内的片段。
    
    Args:
        dp_index_results (dict): 来自 pickle 的数据，结构为 {group: list of trajectory lists}
        lower_bound (int): 残基编号下限
        upper_bound (int): 残基编号上限
        min_length (int): 保留的最小片段长度（可选，避免过短片段）

    Returns:
        dict: 结构为 {group: [segment1, segment2, ...]}，每个 segment 是 list
    """
    valid_segments = {}

    for group, trajectories in dp_index_results.items():
        segments = []
        for traj in trajectories:
            current_segment = []
            for resid in traj:
                if lower_bound <= resid <= upper_bound:
                    current_segment.append(resid)
                else:
                    if len(current_segment) >= min_length:
                        segments.append(current_segment)
                    current_segment = []
            # 检查结尾是否有合法片段
            if len(current_segment) >= min_length:
                segments.append(current_segment)
        valid_segments[group] = segments
    
    return valid_segments

with open("index2.pkl", "rb") as f:
    dp_index_results = pickle.load(f)

valid_segments = extract_valid_segments(dp_index_results)

for group, segs in valid_segments.items():
    print(f"group {group} has {len(segs)} valid segments.")


In [ ]:
import numpy as np

def compute_msd_for_segment(segment, max_dt):
    """
    计算单个片段的 MSD
    """
    segment = np.array(segment)
    n = len(segment)

    msd = []
    for dt in range(1, max_dt + 1):
        displacements = segment[dt:] - segment[:-dt]
        squared_displacement = displacements ** 2
        msd.append(np.mean(squared_displacement))
    return msd

def compute_group_msd(valid_segments, max_dt):
    """
    对每个组的多个片段计算平均 MSD 及误差
    返回结构：{group: {'mean': array, 'sem': array}}
    """
    group_msd_stats = {}

    for group, segments in valid_segments.items():
        all_msds = []
        for segment in segments:
            if len(segment) < 2:
                continue
            msd = compute_msd_for_segment(segment, max_dt)
            all_msds.append(msd)

        # 对齐长度
        min_len = min(len(msd) for msd in all_msds)
        aligned_msds = np.array([msd[:min_len] for msd in all_msds])
        mean_msd = aligned_msds.mean(axis=0)
        sem_msd = aligned_msds.std(axis=0, ddof=1) / np.sqrt(aligned_msds.shape[0])
        group_msd_stats[group] = {
            'mean': mean_msd,
            'sem': sem_msd
        }

    return group_msd_stats

group_msd_stats = compute_group_msd(valid_segments, max_dt=60)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

plt.rcParams.update({
    'font.family': 'Arial',  # 设置字体
    'axes.titlesize': 24,  # 标题字体大小
    'axes.labelsize': 24,  # 坐标轴标签字体大小
    'xtick.labelsize': 24,  # X轴刻度字体大小
    'ytick.labelsize': 24,  # Y轴刻度字体大小
    'legend.fontsize': 24,  # 图例字体大小
    'figure.figsize': (16, 9),  # 图像大小
    'axes.linewidth': 4,  # 坐标轴线宽
    'xtick.major.size': 10, # x 轴主刻度长度
    'ytick.major.size': 10, # y 轴主刻度长度
    'xtick.minor.size': 5, # x 轴次刻度长度
    'ytick.minor.size': 5, # y 轴主刻度长度
    'xtick.major.width': 4, # x 轴主刻度线宽
    'ytick.major.width': 4, # y 轴主刻度线宽
    'xtick.minor.width': 4, # x 轴次刻度线宽
    'ytick.minor.width': 4, # y 轴主刻度线宽
    'legend.fontsize': 32,    # 图例字体大小
    'axes.grid': False,  # 关闭背景网格
})
def compute_group_msd_and_slope(valid_segments, max_dt, fit_range):
    """
    计算每个组的平均 MSD、误差棒、线性拟合斜率（单位：bp²/frame），以及对应的扩散系数 D = slope / 2
    """
    group_stats = {}

    for group, segments in valid_segments.items():
        all_msds = []
        for segment in segments:
            if len(segment) < 2:
                continue
            msd = compute_msd_for_segment(segment, max_dt)
            all_msds.append(msd)

        if len(all_msds) == 0:
            continue

        min_len = min(len(msd) for msd in all_msds)
        aligned_msds = np.array([msd[:min_len] for msd in all_msds])
        mean_msd = aligned_msds.mean(axis=0)
        sem_msd = aligned_msds.std(axis=0, ddof=1) / np.sqrt(aligned_msds.shape[0])

        fit_start, fit_end = fit_range
        x_frames = np.arange(fit_start, fit_end + 1)
        y_msd = mean_msd[fit_start - 1:fit_end]

        slope, intercept, r_value, p_value, std_err = linregress(x_frames, y_msd)
        D = slope / 2
        D_err = std_err / 2

        group_stats[group] = {
            'mean': mean_msd,
            'sem': sem_msd,
            'slope': slope,
            'slope_err': std_err,
            'D': D,
            'D_err': D_err,
            'r2': r_value ** 2
        }

    return group_stats

def plot_msd_with_fit_subplots(group_stats, fit_range, ncols):
    """
    绘制 MSD 曲线与线性拟合，每组一个子图
    - 使用连线代替散点
    - 使用阴影代替误差棒
    """
    n_groups = len(group_stats)
    nrows = (n_groups + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows, ncols, figsize=(18, 13.5))
    axes = axes.flatten()

    for i, (group, stats) in enumerate(group_stats.items()):
        ax = axes[i]
        mean_msd = stats['mean']
        sem_msd = stats['sem']
        frames = np.arange(1, len(mean_msd) + 1)

        # 阴影表示 SEM
        ax.fill_between(frames / 10, 
                        mean_msd - sem_msd, 
                        mean_msd + sem_msd, 
                        color='skyblue', alpha=0.4, label='SEM')

        # 连线表示 MSD
        ax.plot(frames / 10, mean_msd, color='blue', label='MSD (mean)',linewidth=4)

        # 拟合线
        fit_start, fit_end = fit_range
        fit_x = np.arange(fit_start, fit_end + 1)
        y_fit_segment = mean_msd[fit_start - 1:fit_end]
        slope = stats['slope']
        fit_y = slope * fit_x + y_fit_segment[0] - slope * fit_x[0]
        ax.plot(fit_x / 10, fit_y, 'r--', label='Linear fit', zorder=10,linewidth=4)

        ax.set_xlabel("Time (x10$^{-9}$ s)")
        ax.set_ylabel("MSD (x10$^{-6}$kb$^2$)")
        ax.set_title(rf"$\epsilon_{{\mathrm{{HD}}}}$ = {group}", pad=10)

    # 删除多余子图
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    fig.tight_layout()
    plt.savefig('msd_fit_HD.png', dpi=300)
    plt.show()

# === 执行部分 ===

# 这里你需要有 compute_msd_for_segment 和 valid_segments 的定义
# 确保已经定义：
# - compute_msd_for_segment(segment, max_dt)
# - valid_segments = { group_name: [list_of_segments], ... }

group_msd_stats = compute_group_msd_and_slope(
    valid_segments, 
    max_dt=60, 
    fit_range=(30, 50)
)

plot_msd_with_fit_subplots(
    group_msd_stats, 
    fit_range=(30, 50), 
    ncols=3
)


In [ ]:
import numpy as np

# 提取 D 和 D_err 并保存到数组
D_values = []
D_errors = []

for stats in group_msd_stats.values():
    D_values.append(stats['D']*1e4)  # 将 D 转换为 kb²/s
    D_errors.append(stats['D_err']*1e4)  # 将 D_err 转换为 kb²/s

D_values = np.array(D_values)
D_errors = np.array(D_errors)
print(D_values)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection

x = [0, 0.3, 0.5, 0.6, 0.7, 0.9, 1]
y = D_values/1000
error = D_errors/1000

plt.rcParams.update({
    'font.family': 'Arial',  # 设置字体
    'axes.titlesize': 36,  # 标题字体大小
    'axes.labelsize': 36,  # 坐标轴标签字体大小
    'xtick.labelsize': 36,  # X轴刻度字体大小
    'ytick.labelsize': 36,  # Y轴刻度字体大小
    'legend.fontsize': 36,  # 图例字体大小
    'figure.figsize': (7, 6),  # 图像大小
    'axes.linewidth': 4,  # 坐标轴线宽
    'xtick.major.size': 10, # x 轴主刻度长度
    'ytick.major.size': 10, # y 轴主刻度长度
    'xtick.minor.size': 5, # x 轴次刻度长度
    'ytick.minor.size': 0, # y 轴主刻度长度
    'xtick.major.width': 4, # x 轴主刻度线宽
    'ytick.major.width': 4, # y 轴主刻度线宽
    'xtick.minor.width': 4, # x 轴次刻度线宽
    'ytick.minor.width': 4, # y 轴主刻度线宽
    'legend.fontsize': 28,    # 图例字体大小
    'axes.grid': False,  # 关闭背景网格
})

# 使用 color map 将颜色映射到数据点
cmap = plt.colormaps['plasma'] #coolwarm,inferno,magma,plasma
norm = mcolors.Normalize(vmin=min(x), vmax=max(x))
colors = [cmap(norm(value)) for value in x]

fig, ax = plt.subplots()  # 创建一个新的图形和轴

# 创建用于渐变线条的段
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# 创建一个LineCollection对象
lc = LineCollection(segments, colors=colors[:-1], linewidths=4)

# 添加到轴中
ax.add_collection(lc)

# 绘制带有标准误的点
for i in range(len(x)):
    ax.errorbar(x[i], y[i], yerr=error[i], fmt='o', 
                linewidth=4, capsize=16, capthick=20, ecolor=colors[i], 
                color=colors[i], marker='o', markersize=20, 
                markerfacecolor=colors[i], markeredgewidth=4, markeredgecolor='black')

ax.axhline(0.0175, color='#E90F44', linestyle='dashed', linewidth=4)
ax.text(0.0, 0.8, r'$\text{D}_\mathrm{exp.}$', color='#E90F44', fontsize=36)

# 添加图表标题和轴标签
ax.set_xlabel(r'$\epsilon_\text{HD}$')
ax.set_ylabel('D ($10^3$kb$^2$/s)', labelpad=30)

# 设置x轴的刻度为具体的组名
ax.set_xticks(np.arange(0, 1.1, 0.3))
ax.set_yticks(np.arange(0, 8.1, 2))

# x 轴刻度朝内
ax.tick_params(axis='x', which='both', direction='in')
# y 轴刻度朝内
ax.tick_params(axis='y', which='both', direction='in')

#plt.tight_layout()  # 使布局更紧凑
plt.savefig("D_HD.png", dpi=300, bbox_inches='tight')
plt.show()